# Quantum state compression

Classical strings can be compressed to be stored in a more efficient way. The way of quantifing how much a string can be compressed is the Shannon entropy. If we denote a digit of a string $x_i$ and its probability $p(x_i)$ then the Shannon entropy is defined as follows:
$$
H = \sum_{i=1}^n p(x_i)\log p(x_i).
$$
This statement is easily understandable if we think of limit cases:
- the string '11111111...11' is composed always by the same digit. So $p(1)=1$. The entropy is 0, and we can compress the string simply saying '1' n times
- A random string has maximum entropy, and can not be compressed since it is random!

But what about quantum states?
In the following you will find a function to create certain states of $n$ qubits, obtain the statevector from qiskit and then obtain certain metrics:
- The number of non-zero entries in the statevector
- The entanglement along each bipartition, i.e. between $(1, n-1)$, $(2, n-2)$,... $(n-1, 1)$

The states available are the following:
- all plus state: 
    $$
    \frac{|0\rangle + |1\rangle}{\sqrt{2}}\otimes\frac{|0\rangle + |1\rangle}{\sqrt{2}}\otimes\dots\otimes\frac{|0\rangle + |1\rangle}{\sqrt{2}}
    $$
- The GHZ state:
    $$
    \frac{|00\dots0\rangle + |11\dots 1\rangle}{\sqrt{2}}
    $$
- a random state


### BONUS QUESTIONS
- Does it depend on the number of qubits?
- If yes, how?

In [ ]:
# Importing modules
import numpy as np

In [ ]:
# Functions to create the states
def create_allplus(num_qubits=4):
    """
    Create a statevector of the allplus state with `n` qubits
    
    Parameters
    ----------
    n : int
        Number of qubits
        
    Returns
    -------
    np.ndarray
        statevector of the system
    """
    
    statevect_1 = np.ones(2)/np.sqrt(2)
    statevect_tot = np.ones(2)/np.sqrt(2)
    
    for _ in range(num_qubits):
        statevect_tot = np.kron(statevect_tot, statevect_1)
        
    return statevect_tot
    
def create_ghz(num_qubits=4):
    """
    Create a statevector of the GHZ state with `n` qubits
    
    Parameters
    ----------
    n : int
        Number of qubits
        
    Returns
    -------
    np.ndarray
        statevector of the system
    """
    
    ghz = np.zeros(2**num_qubits)
    ghz[0] = 1
    ghz[-1] = 1
    ghz /= np.sqrt(2)
    
    return ghz

def create_random(num_qubits=4):
    """
    Create a statevector of a random state with `n` qubits
    
    Parameters
    ----------
    n : int
        Number of qubits
        
    Returns
    -------
    np.ndarray
        statevector of the system
    """
    
    random = np.random.uniform(0, 1, 2**num_qubits) + 1j*np.random.uniform(0, 1, 2**num_qubits)
    norm = np.sqrt( np.vdot(random, random) )
    random /= norm
    
    return random

In [ ]:
# Evaluation functions

def non_zero(statevect):
    """
    Returns the number of non-zero elements of a statevector
    
    Parameters
    ----------
    statevect : np.ndarray
        statevector of the system
        
    Returns
    -------
    int
        number of non-zero elements of the statevector
    """
    
    non_zero = np.nonzero(statevect)[0]
    
    return len(non_zero)

def entanglement_bond(statevect):
    """
    Returns the entanglement along
    all n-1 bipartitions of the system
    
    Parameters
    ----------
    statevect : np.ndarray
        statevector of the system
        
    Returns
    -------
    array-like of floats
        Entanglement along the bipartitions
    """
    num_sites = int( np.log2(len(statevect)) )
    
    psi_copy=statevect.reshape(*[2 for _ in range(num_sites)])
    
    indeces=np.arange(num_sites)
    entanglement_list = []
    
    for ii in indeces[1:]:
        idx_to_trace = indeces[:ii]

        # COMPUTE THE REDUCED DENSITY MATRIX
        rho=np.tensordot(psi_copy, np.conjugate(psi_copy), axes=(idx_to_trace, idx_to_trace))

        # TRANSFORM INTO A MATRIX. THE NUMBER OF ELEMENTS IS 2^{n_sites-num_traced_idxs }
        rho_dim = int( 2**( (num_sites-len(idx_to_trace)) ) )
        partial_rho = rho.reshape(rho_dim, rho_dim)    
        
        # get eigenvalues of partial_rho
        eigvs, _ = np.linalg.eigh(partial_rho)
        eigvs = eigvs[eigvs>0]

        entanglement = -np.sum( eigvs*np.log(eigvs) )
        entanglement_list.append(entanglement)
    
    return entanglement_list

## It is your turn now! 